In [1]:
import radiomics as rad
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import SimpleITK as sitk

In [2]:
#Import images and read them with SimpleITK
path=r'C:\zach\archive\Dataset_BUSI_with_GT'
categories=["benign","malignant"]
benignmask=[]
malignantmask=[]
benigngray=[]
malignantgray=[]
def read_data():
    for i in categories :
        address = os.path.join(path,i)
        tag=categories.index(i)
        listy=[]
        for j in os.listdir(address):
            img = cv2.imread(os.path.join(address,j),cv2.IMREAD_GRAYSCALE)
            img = sitk.GetImageFromArray(img)
            if 'mask' in j:
                if(not(tag)):
                    listy.append(img)
                else:
                    listy.append(img)
        
            else :
                if(not(tag) and len(listy)!=0):
                    benignmask.append(listy)
                    listy=[]
                elif len(listy)!=0:
                    malignantmask.append(listy)
                    listy=[]
                if(not(tag)):
                    benigngray.append(img)
                else:
                    malignantgray.append(img)
        if(not(tag)):
            benignmask.append(listy)
            listy=[]
        else:
            malignantmask.append(listy)
            listy=[]
    

In [3]:
read_data()

In [4]:
#Extract firstorder and 2D characteristics
def extract_chars(im3,im4):
    imagedata = {}
    s2=[]
    f0=[]
    extractor=rad.featureextractor.RadiomicsFeatureExtractor()
    extractor.disableAllFeatures()
    extractor.enableFeatureClassByName("shape2D")
    extractor.enableFeatureClassByName("firstorder")
    result=extractor.execute(im3,im4,label=255)
    for featureName in result.keys():
            if(("shape2D" in featureName)):
                s2.append((featureName,float(result[featureName])))
            elif(("firstorder" in featureName)):
                f0.append((featureName,float(result[featureName])))
    imagedata["shape2D"] = s2
    imagedata["firstorder"] = f0
    return imagedata

In [5]:
def vectorize_dict(a):
    FeatureVector=[]
    for i in a.values():
        for j in i:
            FeatureVector.append(j[1])
    return FeatureVector

In [6]:
a=[]
for i in range(len(benignmask)):
    for j in range(len(benignmask[i])):
        a.append(vectorize_dict(extract_chars(benigngray[i], benignmask[i][j])))
        

In [12]:
b=[]
for i in range(len(malignantmask)):
    for j in range(len(malignantmask[i])):
        b.append(vectorize_dict(extract_chars(malignantgray[i], malignantmask[i][j])))

In [13]:
benign=np.array(a)
np.save(r'C:\zach\archive\FeatureVectors\BenignVectors',benign)

In [14]:
malignant=np.array(b)
np.save(r'C:\zach\archive\FeatureVectors\MalignantVectors',malignant)